In [114]:
# Загружаем библиотеки
import pandas as pd # Датафреймы pandas
from sklearn.feature_extraction.text import TfidfVectorizer # Модель TF-IDF
from sklearn.ensemble import RandomForestRegressor # Модель случайного леса
from sklearn.model_selection import train_test_split # Разделение выборки на обучающую и тестовую
from sklearn.metrics import mean_squared_error # Средняя квадратичная ошибка

#### Загрузка данных

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

#### Обработка данных

In [3]:
# В списке жанров заменим разделители на пробелы, удалим тире, преобразуем к нижнему регистру
movies['genres'] = movies['genres'].apply(lambda string: ' '.join(string.replace('-', '').split('|')).lower())

In [90]:
# Берём столбцы с номерами пользователей, фильмов и теги, объединяем с таблицей фильмов по номеру фильма
movies_tagged = movies.join(tags[['userId', 'movieId', 'tag']].set_index('movieId'), on = 'movieId')
# Преобразуем теги: удалим все пробелы, приведём к нижнему регистру
movies_tagged['tag'] = movies_tagged['tag'].apply(lambda string: str(string).replace('r:', '').replace('(', '').replace(')', '').replace('-', '').lower())

In [91]:
# Удалим строки с пустыми значениями
movies_tagged.dropna(inplace = True)
# Создадим столбцы для среднего рейтинга фильма и среднего рейтинга пользователя
movies_tagged['movie_rating'] = 0
movies_tagged['user_rating'] = 0
# Переставим столбцы местами для удобства просмотра
movies_tagged = movies_tagged[['movieId', 'movie_rating','title', 'userId', 'user_rating', 'genres', 'tag']]
# Сбросим индекс
movies_tagged = movies_tagged.reset_index(drop=True)

#### Расчёт средних рейтингов

In [92]:
# Средний рейтинг фильма по всем пользователям
movies_tagged['movie_rating'] = movies_tagged['movieId'].apply(lambda movieId: ratings[ratings['movieId'] == movieId]['rating'].mean())

In [93]:
# Средний рейтинг пользователя по всем фильмам
movies_tagged['user_rating'] = movies_tagged['userId'].apply(lambda userId: ratings[ratings['userId'] == userId]['rating'].mean())

In [94]:
movies_tagged['userId'] = movies_tagged['userId'].astype('int64')

#### Модель TF-IDF

In [98]:
# Вызываем модель TF-IDF и обучаем её, преобразуем к матрице
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(list(movies_tagged['genres'] + ' ' + movies_tagged['tag']))

#### Модель случайного леса и оценка точности

In [145]:
# Разделяем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix[:500], movies_tagged['movie_rating'][:500], random_state = 42)

In [146]:
# Используем модель случайного леса
model = RandomForestRegressor(random_state = 42)
# Обучаем модель на обучающей выборке
model.fit(X_train, y_train)
print('Модель для среднего рейтинга фильма по всем пользователям:')
print(f'- показатель точности {model.score(X_test, y_test)}')
print(f'- RMSE {mean_squared_error(y_test, model.predict(X_test), squared = False)}')

Модель для среднего рейтинга фильма по всем пользователям:
- показатель точности 0.5054373261548336
- RMSE 0.3016470116748055


In [147]:
# Разделяем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix[:500], movies_tagged['user_rating'][:500], random_state = 42)

In [149]:
# Используем модель случайного леса
model = RandomForestRegressor(random_state = 42)
# Обучаем модель на обучающей выборке
model.fit(X_train, y_train)
print('Модель для среднего рейтинга пользователя по всем фильмам:')
print(f'- показатель точности {model.score(X_test, y_test)}')
print(f'- RMSE {mean_squared_error(y_test, model.predict(X_test), squared = False)}')

Модель для среднего рейтинга пользователя по всем фильмам:
- показатель точности 0.6055448809403072
- RMSE 0.33806865705775596


Полученная точность очень низкая! Пробовал слить все слова в каждом теге в одно длинное слово без пробелов для учёта уникальности тегов, но это не помогло повысить точность.